# Sort by camera
## 12/15/21
Group all data in the 620 benthic concepts from 2000-2019 by which camera collected the image

In [1]:
import os
import sys
import glob
import datetime
import requests
import json
import progressbar
import subprocess
import numpy as np
import pandas as pd
from fathomnet.api import images
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from scipy.spatial.distance import braycurtis

This will be sort of hacky as there currently is not a field in FathomNet for camera used to collect the image. But there are these date ranges for cameras deployed on each ROV.

| rov_name | camera_name | start_date | end_date | fov_width_pixels | fov_height_pixels |
| --- | --- | --- | --- | --- | --- |  
| Doc Ricketts | Ikegami HDL40  | 2/10/2009 12:00:00 AM | 12/31/2019 11:59:00 PM | 1920 | 1080 |
| Tiburon | Panasonic 3-chip | 10/29/1996 12:00:00 AM | 9/23/2005 11:59:00 PM | 720 | 480 |
| Tiburon | Ikegami HDL40 | 10/4/2005 12:00:00 AM | 12/20/2007 11:59:00 PM | 1920 | 1080 |
| Ventana | Sony 3-chip | 8/25/1988 12:00:00 AM | 8/25/1999 11:59:00 PM | 720 | 480 |
| Ventana | Sony HDTV | 9/3/1999 12:00:00 AM | 7/9/2007 11:59:00 PM | 1920 | 1080 |
| Ventana | Sony 3-chip | 7/11/2007 12:00:00 AM | 7/31/2007 11:59:00 PM | 720 | 480 |
| Ventana | Ikegami HDL40 | 8/2/2007 12:00:00 AM | 12/31/2019 11:59:00 PM | 1920 | 1080 |

First make the table into a dictonary and then load in the COCO json dataset.

In [8]:
cams = {'Doc%20Ricketts': {'ikegami': ['2009-02-10', '2019-12-31']},
        'Tiburon': {'panasonic': ['1996-10-29', '2005-09-23'],
                    'ikegami': ['2005-10-04', '2007-12-20']},
        'Ventana': {'sonyChip': ['2007-07-11', '2007-07-31'],
                    'sonyHD': ['1999-09-03', '2007-07-09'],
                    'ikegami': ['2008-08-02', '2019-12-31']}
       }

In [2]:
with open('/Users/eorenstein/Documents/fathomnet/duds/datasets/dataset.json', 'r') as ff:
    anns = json.load(ff)
    ff.close()

Now pull out the url strings that are needed to retrieve the entire FathomNet AImageDTO object. Then split into separate lists based on vehicle.

In [18]:
urls = [(line['coco_url'], line['coco_url'].split('/')[-4]) for line in anns['images']]
urls = list(set(urls))  # remove any duplicates 

vehicles = {}
vehicles['vent'] = [line[0] for line in urls if line[1] == 'Ventana']
vehicles['rick'] = [line[0] for line in urls if line[1] == 'Doc%20Ricketts']
vehicles['tib'] = [line[0] for line in urls if line[1] == 'Tiburon']

print('Ventana =', len(vehicles['vent']))
print('Ricketts =', len(vehicles['rick']))
print('Tiburon =', len(vehicles['tib']))

Ventana = 6757
Ricketts = 18468
Tiburon = 3166


Iterate over the urls to get the rest of the information we need.

In [4]:
for vv in vehicles.keys():
    for url in progressbar.progressbar(vehicles[vv]):
            tmp = images.find_by_url(url) 

100% (10 of 10) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
